# Tutorial completion Notebook

In [ ]:
#required imports
import numpy as np, matplotlib.pyplot as plt
import sep, astropy.io.fits as fits
from matplotlib import rcParams

%matplotlib inline

In [ ]:
#import fits formatted data to array
rawData = fits.open("./galaxies.fits")
data = fits.getdata("./galaxies.fits")

#display image with matplotlib
mean, std = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=mean-std, vmax=mean+std, origin='lower')
plt.colorbar();

In [ ]:
#background removal processing
#I'll come back to find out about the parameters in use here
bkg = sep.Background(data, bw=64, bh=64, fw=3, fh=3)

# get a "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()

# show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

In [ ]:
#background removal cont'd
# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

# show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();


# subtract the background
data_sub = data - bkg

In [ ]:
#get the objects
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

# how many objects were detected
#print(len(objects))

from matplotlib.patches import Ellipse

#plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    
fig.set_facecolor("white")
fig.savefig("test identification.png", bbox_inches = "tight", dpi = 300)

In [ ]:
#aperture photmetry
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))